In [6]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools
using StaticArrays

using Profile
using PProf
using ProfileSVG

import Base: show

In [3]:
precision = datavar(:precision, Gamma{Float64})
mean      = datavar(:mean, NormalMeanPrecision{Float64})

DataVariable{Subject{Message{NormalMeanPrecision{Float64}}, AsapScheduler},NormalMeanPrecision{Float64}}(:mean, Subject(Message{NormalMeanPrecision{Float64}}, AsapScheduler), ReactiveMP.DataVariableProps(nothing))

In [7]:
node = GaussianMeanPrecisionNode(factorisation = SA[ SA[1], SA[2], SA[3] ])

Node{NormalMeanPrecision{Float64},3,SArray{Tuple{3},SArray{Tuple{1},Int64,1,1},1,3}}(NodeVariable[mean, precision, value], SArray{Tuple{1},Int64,1,1}[[1], [2], [3]])

In [8]:
y = datavar(:y, Float64)

DataVariable{Subject{Message{Float64}, AsapScheduler},Float64}(:y, Subject(Message{Float64}, AsapScheduler), ReactiveMP.DataVariableProps(nothing))